<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_02_2_multi_prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 2: Code Generation**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 2 Material

* Part 2.1: Prompting for Code Generation [[Video]](https://www.youtube.com/watch?v=HVId6kYKKgQ) [[Notebook]](t81_559_class_02_1_dev.ipynb)
* **Part 2.2: Handling Revision Prompts** [[Video]](https://www.youtube.com/watch?v=APpV46tplXA) [[Notebook]](t81_559_class_02_2_multi_prompt.ipynb)
* Part 2.3: Using a LLM to Help Debug [[Video]](https://www.youtube.com/watch?v=VPqSNb38QK0) [[Notebook]](t81_559_class_02_3_llm_debug.ipynb)
* Part 2.4: Tracking Prompts in Software Development [[Video]](https://www.youtube.com/watch?v=oUFUuYfvXZU) [[Notebook]](t81_559_class_02_4_software_eng.ipynb)
* Part 2.5: Limits of LLM Code Generation [[Video]](https://www.youtube.com/watch?v=dKtRI0LZSyY) [[Notebook]](t81_559_class_02_5_code_gen_limits.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:

import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set COLAB to True
COLAB = True
print("Note: using Google CoLab")

# OpenAI Secrets
if COLAB:
    # Path to the file containing the API key
    api_key_path = '/content/drive/My Drive/openai_api_key.txt'

    # Read the API key from the file
    with open(api_key_path, 'r') as file:
        os.environ["OPENAI_API_KEY"] = file.read().strip()

# Print the API key to verify (optional, for debugging purposes)
print(os.environ["OPENAI_API_KEY"])

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai

# 2.2: Handling Revision Prompts

Previously, we just sent one prompt to the LLM, which generated code. It is possible to perform this code more conversationally. In this module, we will see how to converse with the LLM to request changes to outputted code and even help the LLM to produce a more accurate model.

We will also see that it might be beneficial to recreate your conversation as one single prompt that generates the final result. Keeping track of one prompt, rather than a conversation, that created your final code is more maintainable.

## Conversational Code Generation

We will introduce a more advanced code generation function that allows you to start the conversation to generate code and follow up with additional prompts if needed.

In future modules, we will see how to create chatbots similar to this one. We will use the code I provided to generate your code for now. This generator uses a system prompt that requests that the generated code conform to the following:

* Imports should be sorted
* Code should conform to PEP-8 formatting
* Do not mix uncompilable notes with code
* Add comments

In [2]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import PromptTemplate
from IPython.display import display_markdown

MODEL = 'gpt-4o-mini'
TEMPLATE = """The following is a friendly conversation between a human and an
AI to generate Python code. If you have notes about the code, place them before
the code. Any nots about execution should follow the code. If you do mix any
notes with the code, make them comments. Add proper comments to the code.
Sort imports and follow PEP-8 formatting.

Current conversation:
{history}
Human: {input}
Code Assistant:"""
PROMPT_TEMPLATE = PromptTemplate(input_variables=["history", "input"], template=TEMPLATE)

def start_conversation():
    # Initialize the OpenAI LLM with your API key
    llm = ChatOpenAI(
        model=MODEL,
        temperature=0.0,
        n=1
    )

    # Initialize memory and conversation
    memory = ConversationBufferWindowMemory()
    conversation = ConversationChain(
        prompt=PROMPT_TEMPLATE,
        llm=llm,
        memory=memory,
        verbose=False
    )

    return conversation

def generate_code(conversation, prompt):
    print("Model response:")
    output = conversation.invoke(prompt)
    display_markdown(output['response'], raw=True)


## First Attempt at an XOR Approximator

We will construct a prompt that requests the LLM to generate a PyTorch neural network to approximate the [Exclusive Or](https://en.wikipedia.org/wiki/Exclusive_or). The truth table for the Exclusive Or (XOR) function is provided here:

```
0 XOR 0 = 0
1 XOR 0 = 1
0 XOR 1 = 1
1 XOR 1 = 0
```

If given data, neural networks can learn to approximate functions, so let's create a PyTorch neural network to approximate the XOR function.

In [3]:
conversation = start_conversation()
generate_code(conversation, """Write Python code to learn the XOR function with PyTorch.""")

<ipython-input-2-860a868a87f1>:29: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory()
<ipython-input-2-860a868a87f1>:30: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


Model response:


Sure! Below is a Python code snippet that demonstrates how to learn the XOR function using PyTorch. The code includes comments to explain each part of the process.

```python
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR dataset
# Input: 2 features (x1, x2), Output: 1 target (y)
data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
labels = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# Define a simple feedforward neural network
class XORModel(nn.Module):
    def __init__(self):
        super(XORModel, self).__init__()
        # Input layer to hidden layer
        self.hidden = nn.Linear(2, 2)
        # Hidden layer to output layer
        self.output = nn.Linear(2, 1)
        # Activation function
        self.activation = nn.Sigmoid()

    def forward(self, x):
        # Pass input through the hidden layer
        hidden_output = self.activation(self.hidden(x))
        # Pass hidden layer output to the output layer
        output = self.output(hidden_output)
        return self.activation(output)

# Initialize the model, loss function, and optimizer
model = XORModel()
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.1)  # Stochastic Gradient Descent

# Training the model
num_epochs = 10000  # Number of epochs for training
for epoch in range(num_epochs):
    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass: compute predicted outputs
    outputs = model(data)
    # Compute the loss
    loss = criterion(outputs, labels)
    # Backward pass: compute gradients
    loss.backward()
    # Update weights
    optimizer.step()

    # Print loss every 1000 epochs
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model
with torch.no_grad():
    test_data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
    predictions = model(test_data)
    print("Predictions:")
    print(predictions.round())  # Round the predictions to get binary output
```

### Notes on Execution:
- Ensure you have PyTorch installed in your Python environment. You can install it using pip:
  ```bash
  pip install torch
  ```
- The model is trained for 10,000 epochs, which should be sufficient for it to learn the XOR function. You can adjust the number of epochs or the learning rate if needed.
- The output predictions are rounded to get binary values (0 or 1) for the XOR function.

# Requesting a Change to Generated Code

If you've taken my other course, you will know I prefer PyTorch sequences over extending the nn.Module class, at least for simple neural networks like an XOR approximator. LLMs do not share this opinion. However, the LLM will gladly humor me and generate a sequence. Here, I provide an additional prompt to request this rather than resubmitting a modified version of my first prompt.

In [4]:
generate_code(conversation, """Could you make use of a PyTorch sequence rather than a nn.Module class?""")

Model response:


Certainly! Below is the modified Python code that uses a PyTorch `nn.Sequential` to define the model instead of creating a custom `nn.Module` class. The code includes comments to explain each part of the process.

```python
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR dataset
# Input: 2 features (x1, x2), Output: 1 target (y)
data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
labels = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# Define a simple feedforward neural network using nn.Sequential
model = nn.Sequential(
    nn.Linear(2, 2),  # Input layer to hidden layer
    nn.Sigmoid(),     # Activation function for hidden layer
    nn.Linear(2, 1),  # Hidden layer to output layer
    nn.Sigmoid()      # Activation function for output layer
)

# Initialize the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.1)  # Stochastic Gradient Descent

# Training the model
num_epochs = 10000  # Number of epochs for training
for epoch in range(num_epochs):
    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass: compute predicted outputs
    outputs = model(data)
    # Compute the loss
    loss = criterion(outputs, labels)
    # Backward pass: compute gradients
    loss.backward()
    # Update weights
    optimizer.step()

    # Print loss every 1000 epochs
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model
with torch.no_grad():
    test_data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
    predictions = model(test_data)
    print("Predictions:")
    print(predictions.round())  # Round the predictions to get binary output
```

### Notes on Execution:
- Ensure you have PyTorch installed in your Python environment. You can install it using pip:
  ```bash
  pip install torch
  ```
- The model is trained for 10,000 epochs, which should be sufficient for it to learn the XOR function. You can adjust the number of epochs or the learning rate if needed.
- The output predictions are rounded to get binary values (0 or 1) for the XOR function.

# Testing the Generated Code

LLMs are not overachievers; they will implement the code you ask for and not provide much more. When we run the XOR approximator's first version, the results are only sometimes accurate, especially if we run the program multiple times.

In [5]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR network using a sequential container
model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Sigmoid(),
    nn.Linear(2, 1),
    nn.Sigmoid()
)

# Initialize the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# Training data for XOR
data = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
labels = torch.tensor([[0.0], [1.0], [1.0], [0.0]])

# Train the model
for epoch in range(10000):
    # Forward pass: Compute predicted y by passing x to the model
    pred = model(data)

    # Compute and print loss
    loss = criterion(pred, labels)
    if epoch % 1000 == 0:
        print(f'Epoch {epoch} Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test the model
with torch.no_grad():
    test_pred = model(data)
    print("Predicted values:")
    print(test_pred)

Epoch 0 Loss: 0.25302624702453613
Epoch 1000 Loss: 0.24863247573375702
Epoch 2000 Loss: 0.24629078805446625
Epoch 3000 Loss: 0.2398288995027542
Epoch 4000 Loss: 0.22486700117588043
Epoch 5000 Loss: 0.20344685018062592
Epoch 6000 Loss: 0.18549028038978577
Epoch 7000 Loss: 0.1696184277534485
Epoch 8000 Loss: 0.1411036103963852
Epoch 9000 Loss: 0.08170422911643982
Predicted values:
tensor([[0.1506],
        [0.8082],
        [0.8080],
        [0.2405]])


If you receive an error or the output is not exactly what you like, it is effective to provide that output and any errors to the LLM. Here, we provide the output and ask the LLM if that seems correct. Sometimes, the LLM may insist that the output is correct, so you must "debate" the LLM, providing additional details.

In [6]:
generate_code(conversation, """The output was:

Predicted values:
tensor([[0.4843],
        [0.5800],
        [0.4278],
        [0.4623]])

Are you sure that is correct?
""")

Model response:


It seems that the model may not have fully learned the XOR function, which is a common issue when training neural networks, especially with simple architectures. The XOR function is not linearly separable, and sometimes a more complex model or different training parameters are needed to achieve better results.

Let's ensure that the model is correctly set up and consider a few adjustments. Below is the revised code with a slightly more complex architecture (adding an additional hidden layer) to improve the learning capability of the model.

```python
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR dataset
# Input: 2 features (x1, x2), Output: 1 target (y)
data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
labels = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# Define a simple feedforward neural network using nn.Sequential
model = nn.Sequential(
    nn.Linear(2, 4),  # Input layer to first hidden layer
    nn.Sigmoid(),     # Activation function for first hidden layer
    nn.Linear(4, 2),  # First hidden layer to second hidden layer
    nn.Sigmoid(),     # Activation function for second hidden layer
    nn.Linear(2, 1),  # Second hidden layer to output layer
    nn.Sigmoid()      # Activation function for output layer
)

# Initialize the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.1)  # Stochastic Gradient Descent

# Training the model
num_epochs = 10000  # Number of epochs for training
for epoch in range(num_epochs):
    # Zero the gradients
    optimizer.zero_grad()
    # Forward pass: compute predicted outputs
    outputs = model(data)
    # Compute the loss
    loss = criterion(outputs, labels)
    # Backward pass: compute gradients
    loss.backward()
    # Update weights
    optimizer.step()

    # Print loss every 1000 epochs
    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model
with torch.no_grad():
    test_data = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
    predictions = model(test_data)
    print("Predictions:")
    print(predictions.round())  # Round the predictions to get binary output
```

### Notes on Execution:
- Ensure you have PyTorch installed in your Python environment. You can install it using pip:
  ```bash
  pip install torch
  ```
- The model architecture has been modified to include an additional hidden layer with 4 neurons, which should help the model learn the XOR function more effectively.
- The model is still trained for 10,000 epochs, which should be sufficient. You can adjust the number of epochs or the learning rate if needed.
- The output predictions are rounded to get binary values (0 or 1) for the XOR function. After running this code, you should see predictions closer to the expected output of the XOR function.

## Test the Improved Version

We now receive much more accurate output when we test the neural network provided.

In [7]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR network using a sequential container
model = nn.Sequential(
    nn.Linear(2, 4),  # Increased the number of neurons in the hidden layer
    nn.Sigmoid(),
    nn.Linear(4, 1),
    nn.Sigmoid()
)

# Initialize the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)  # Changed to Adam optimizer

# Training data for XOR
data = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
labels = torch.tensor([[0.0], [1.0], [1.0], [0.0]])

# Train the model
for epoch in range(20000):  # Increased the number of epochs
    # Forward pass: Compute predicted y by passing x to the model
    pred = model(data)

    # Compute and print loss
    loss = criterion(pred, labels)
    if epoch % 1000 == 0:
        print(f'Epoch {epoch} Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Test the model
with torch.no_grad():
    test_pred = model(data)
    print("Predicted values:")
    print(test_pred)

Epoch 0 Loss: 0.2580437660217285
Epoch 1000 Loss: 5.0754984840750694e-05
Epoch 2000 Loss: 1.4872541214572266e-05
Epoch 3000 Loss: 6.542353730765171e-06
Epoch 4000 Loss: 3.3518126656417735e-06
Epoch 5000 Loss: 1.848135752879898e-06
Epoch 6000 Loss: 1.0599346751405392e-06
Epoch 7000 Loss: 6.216667429725931e-07
Epoch 8000 Loss: 3.69443455383589e-07
Epoch 9000 Loss: 2.212603931184276e-07
Epoch 10000 Loss: 1.331342218691134e-07
Epoch 11000 Loss: 8.035284082552607e-08
Epoch 12000 Loss: 4.859315794192298e-08
Epoch 13000 Loss: 2.94252586741095e-08
Epoch 14000 Loss: 1.784731296083919e-08
Epoch 15000 Loss: 1.083044054439597e-08
Epoch 16000 Loss: 6.579485400948215e-09
Epoch 17000 Loss: 4.005686449204404e-09
Epoch 18000 Loss: 2.4488615579798534e-09
Epoch 19000 Loss: 1.5028531752392382e-09
Predicted values:
tensor([[3.1293e-05],
        [9.9997e-01],
        [9.9997e-01],
        [3.6038e-05]])


## Combining the Conversation into a Single Prompt

We should combine this entire conversation into a single prompt, especially if we wish to save the prompt along with the code. We can request the LLM to create this combined prompt for us.

In [8]:
generate_code(conversation, """Okay, that is great, can you suggest a single
prompt that would have resulted in this last code output?""")

Model response:


Certainly! A single prompt that would have led to the last code output could be:

"Write Python code using PyTorch to learn the XOR function with a feedforward neural network that includes two hidden layers. Ensure to use `nn.Sequential` for model definition, and include comments to explain each part of the code. The model should be trained for 10,000 epochs, and the output predictions should be rounded to get binary values (0 or 1)."

This prompt clearly specifies the requirements for the model architecture, the use of `nn.Sequential`, the training duration, and the need for comments, which would guide the generation of the desired code.

The LLM's attempt at a consoldated prompt is incomplete. It skips several important details and does not provide precise requirements. I will manually make some improvements, which you can see here.

In [9]:
# Start a new conversation
conversation = start_conversation()
generate_code(conversation, """
Can you provide Python code using PyTorch to effectively learn the XOR function
with 4 hidden neurons, using the Adam optimizer, and 20K training epochs?
Use a sequence not a nn.Module class.""")

Model response:


Sure! Below is the Python code that uses PyTorch to learn the XOR function with 4 hidden neurons, the Adam optimizer, and 20,000 training epochs. The code is structured as a sequence rather than using a `nn.Module` class.

```python
import torch
import torch.nn as nn
import torch.optim as optim

# Set the random seed for reproducibility
torch.manual_seed(0)

# Define the XOR input and output
X = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# Create a sequential model
model = nn.Sequential(
    nn.Linear(2, 4),  # Input layer to hidden layer with 4 neurons
    nn.ReLU(),        # Activation function
    nn.Linear(4, 1),  # Hidden layer to output layer
    nn.Sigmoid()      # Activation function for output
)

# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer

# Training loop
num_epochs = 20000
for epoch in range(num_epochs):
    # Forward pass: compute predicted y by passing x to the model
    y_pred = model(X)

    # Compute and print loss
    loss = criterion(y_pred, y)
    
    # Zero gradients, backward pass, and optimize
    optimizer.zero_grad()  # Clear previous gradients
    loss.backward()        # Backpropagation
    optimizer.step()       # Update weights

    # Print loss every 2000 epochs
    if (epoch + 1) % 2000 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Test the model
with torch.no_grad():
    test_output = model(X)
    predicted = (test_output > 0.5).float()  # Convert probabilities to binary output
    print("Predicted outputs:")
    print(predicted)
```

### Notes on Execution:
- Ensure you have PyTorch installed in your Python environment. You can install it using pip if you haven't done so already: `pip install torch`.
- The model is trained for 20,000 epochs, which may take some time depending on your hardware.
- The output will show the loss at every 2000 epochs, and finally, it will print the predicted outputs for the XOR function.
- The predicted outputs will be in binary format (0 or 1) based on the learned model.

## Test the Final Prompt

Now, we test the final prompt. My prompt produces an acceptable result, but there are some opportunities for improvement. You can specify the exact format for the output. For example, sometimes code is generated to round the results, but other times it is not.

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the XOR inputs and outputs
inputs = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float)
targets = torch.tensor([[0], [1], [1], [0]], dtype=torch.float)

# Define the model using a sequential container
model = nn.Sequential(
    nn.Linear(2, 4),  # Input layer to hidden layer with 4 neurons
    nn.ReLU(),        # ReLU activation function
    nn.Linear(4, 1),  # Hidden layer to output layer
    nn.Sigmoid()      # Sigmoid activation function for binary output
)

# Define the loss function and the optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer with learning rate of 0.01

# Training loop
for epoch in range(20000):  # 20,000 training epochs
    optimizer.zero_grad()   # Clear gradients for each training step
    outputs = model(inputs)  # Forward pass: compute predicted outputs by passing inputs to the model
    loss = criterion(outputs, targets)  # Compute loss
    loss.backward()  # Backward pass: compute gradient of the loss with respect to model parameters
    optimizer.step()  # Perform a single optimization step (parameter update)

    if (epoch + 1) % 1000 == 0:
        print(f'Epoch [{epoch + 1}/20000], Loss: {loss.item():.4f}')

# Testing the model
with torch.no_grad():  # Context-manager that disabled gradient calculation
    predicted = model(inputs).round()  # Forward pass and rounding off to get predictions
    print(f'Predicted tensor: {predicted}')
    print(f'Actual tensor: {targets}')

Epoch [1000/20000], Loss: 0.0019
Epoch [2000/20000], Loss: 0.0004
Epoch [3000/20000], Loss: 0.0002
Epoch [4000/20000], Loss: 0.0001
Epoch [5000/20000], Loss: 0.0000
Epoch [6000/20000], Loss: 0.0000
Epoch [7000/20000], Loss: 0.0000
Epoch [8000/20000], Loss: 0.0000
Epoch [9000/20000], Loss: 0.0000
Epoch [10000/20000], Loss: 0.0000
Epoch [11000/20000], Loss: 0.0000
Epoch [12000/20000], Loss: 0.0000
Epoch [13000/20000], Loss: 0.0000
Epoch [14000/20000], Loss: 0.0000
Epoch [15000/20000], Loss: 0.0000
Epoch [16000/20000], Loss: 0.0000
Epoch [17000/20000], Loss: 0.0000
Epoch [18000/20000], Loss: 0.0000
Epoch [19000/20000], Loss: 0.0000
Epoch [20000/20000], Loss: 0.0000
Predicted tensor: tensor([[0.],
        [1.],
        [1.],
        [0.]])
Actual tensor: tensor([[0.],
        [1.],
        [1.],
        [0.]])
